In [2]:
import numpy as np
import torch
bikes_np = np.loadtxt("hour-fixed.csv", dtype=np.float32, delimiter=",", skiprows=1, converters={1: lambda x: float(x[8:10])})
bikes = torch.from_numpy(bikes_np)
bikes

tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 3.0000e+00, 1.3000e+01,
         1.6000e+01],
        [2.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 8.0000e+00, 3.2000e+01,
         4.0000e+01],
        [3.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 5.0000e+00, 2.7000e+01,
         3.2000e+01],
        ...,
        [1.7377e+04, 3.1000e+01, 1.0000e+00,  ..., 7.0000e+00, 8.3000e+01,
         9.0000e+01],
        [1.7378e+04, 3.1000e+01, 1.0000e+00,  ..., 1.3000e+01, 4.8000e+01,
         6.1000e+01],
        [1.7379e+04, 3.1000e+01, 1.0000e+00,  ..., 1.2000e+01, 3.7000e+01,
         4.9000e+01]])

In [3]:
bikes.shape, bikes.stride()


(torch.Size([17520, 17]), (17, 1))

In [4]:
bikes_daily = bikes.view(-1, 24, bikes.shape[1])
bikes_daily.shape, bikes_daily.stride()

(torch.Size([730, 24, 17]), (408, 17, 1))

In [5]:
bikes_daily = bikes_daily.transpose(1, 2)
bikes_daily.shape, bikes_daily.stride()

(torch.Size([730, 17, 24]), (408, 1, 17))

In [6]:
first = bikes[:24].long()
weather_onehot = torch.zeros(first.shape[0], 4)
first[:,9]

tensor([1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2])

In [7]:
weather_onehot.scatter_(dim=1, index=first[:,9].unsqueeze(1) - 1, value=1.0)

tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]])

In [8]:
torch.cat((bikes[:24], weather_onehot), 1)[:1]

tensor([[ 1.0000,  1.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  6.0000,
          0.0000,  1.0000,  0.2400,  0.2879,  0.8100,  0.0000,  3.0000, 13.0000,
         16.0000,  1.0000,  0.0000,  0.0000,  0.0000]])

In [9]:
weather_daily_onehot = torch.zeros(bikes_daily.shape[0], 4,
bikes_daily.shape[2])
weather_daily_onehot.shape

torch.Size([730, 4, 24])

In [10]:
weather_daily_onehot.scatter_(1, bikes_daily[:,9,:].long().unsqueeze(1) - 1,
     1.0)
weather_daily_onehot.shape

torch.Size([730, 4, 24])

In [0]:
bikes_daily = torch.cat((bikes_daily, weather_daily_onehot), dim=1)

In [0]:
bikes_daily[:, 9, :] = (bikes_daily[:, 9, :] - 1.0) / 3.0

In [0]:
tmp = bikes_daily[:, 10, :]
min = torch.min(tmp)
max = torch.max(tmp)
bikes_daily[:, 10, :] = (bikes_daily[:, 10, :] - min) / (max -
min)

In [0]:
tmp = bikes_daily[:, 10, :]
bikes_daily[:, 10, :] = (bikes_daily[:, 10, :] - torch.mean(tmp)) / torch.std(tmp)